the function func_run contains the main body of code which reads in data, solves Geo-I, and p mpc-H. it takes as an input the number of user.


In [1]:
import pandas as pd
import numpy as np
import math 
from casadi import *
from pylab import *
import os 
from datetime import datetime, timedelta
import pymap3d # python3 -m pip install pymap3d
import random
import pickle
from scipy.special import lambertw
from scipy import stats
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output, save
from bokeh.layouts import gridplot, row, column
from bokeh.models import Range1d
output_notebook() 



Loading BokehJS ...

In [2]:
nbuf = 30

# Functions to use

In [3]:
def system_update(system,x,y,b):
    dict = {'X': x, 'Y': y, 'N': b}
    system = system.append(dict, ignore_index = True).drop(index=[0]).reset_index(drop=True)
    return system

def privacy(system):
    n = system['N'].sum()
    if n == 0:
        barx = 0
        bary = 0
        priv = 0
    else:
        barx = np.dot(system['X'], system['N'])/n
        bary = np.dot(system['Y'], system['N'])/n
        #priv = np.sqrt(np.dot((system['X'] - barx)**2 + (system['Y'] - bary)**2, system['N'])/n) 
        priv = np.dot(np.sqrt((system['X'] - barx)**2 + (system['Y'] - bary)**2), system['N'])/n 
    return barx, bary, priv

def utility(x,y):
    return np.sqrt(x**2 + y**2)

def reset_system(system):
    n = len(system)
    X = np.array([0]*n)
    Y = np.array([0]*n)
    N = np.array([0]*n)

    system['X'] = X
    system['Y'] = Y
    system['N'] = N
    return system



def GeoI_LPPM(x,y,epsilon):
    theta_GeoI = random.uniform(0,2*math.pi)
    r_GeoI = -1/epsilon*(np.real(lambertw((random.random()-1)/math.exp(1),k=-1))+1);
    x_ctrl = x + r_GeoI*math.cos(theta_GeoI)
    y_ctrl = y + r_GeoI*math.sin(theta_GeoI)
    return x_ctrl, y_ctrl, r_GeoI 
    

## Symbolic functions


In [4]:
from sys import path

# Control
u = MX.sym("u")
v = MX.sym("v")
b = MX.sym("b")

# State
x = MX.sym("x",nbuf)
y = MX.sym("y",nbuf)
nx = MX.sym("nx",nbuf)

# Matrices
A = np.zeros((nbuf, nbuf))
B = np.zeros((nbuf, 1))
for i in range(nbuf-1):
    A[i,i+1] = 1

B[nbuf-1,0] = 1
A.tolist()
B.tolist()
A = DM(A)
B = DM(B)

# Dynamics
xplus = A@x + B@u
yplus = A@y + B@v
nplus = A@nx + B@b

# Discrete time dynamics function
F = Function('F', [x,u],[xplus])

#Privacity function
O = DM(np.ones((1,nbuf)))
xbar = (O@(x*nx))/((O@nx)+1e-8)
ybar = O@(y*nx)/((O@nx)+1e-8)
P = (O@( sqrt((x-xbar)**2 + (y-ybar)**2)*nx) )/((O@nx)+1e-8)
J = Function('J', [x,y,nx],[xbar, ybar, P],['x','y','nx'],['xbar', 'ybar','P'])

# Test privacy functions
x1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
y1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
nx1 = [1]*nbuf

## MPC function

In [5]:
#Defining mpc function
def solve_mpc(xs,ys,us,bs,horizon,nbuf,util_bound):
    nt = len(xs)
    x_mpc = []
    y_mpc = []
    U_mpc = []
    V_mpc = []
    barx_mpc = []
    bary_mpc = []
    priv_mpc = []
    util_mpc = []
    time_mpc = []
    X0 = np.array([0]*nbuf).tolist()
    Y0 = np.array([0]*nbuf).tolist()
    N0 = np.array([0]*nbuf).tolist()

    system_opt = pd.DataFrame()
    system_opt['X'] = X0
    system_opt['Y'] = Y0
    system_opt['N'] = N0

    for tk in range(0,nt-horizon):#nt-horizon):

        if bs[tk] == 0:
            U_mpc += [0]
            V_mpc += [0]
            x_mpc += [xs[tk]]
            y_mpc += [ys[tk]]
            system_opt = system_update(system_opt, xs[tk], ys[tk], 0)
            barxt, baryt, privt = privacy(system_opt)
            barx_mpc += [barxt] 
            bary_mpc += [baryt] 
            priv_mpc += [privt] 
            # Compute utility
            util_mpc += [-1]
            time_mpc += [0]
            #system_opt = system_update(system_opt, xs[tk], ys[tk], 0)

        else:
            time_0= time.time()
            X0 = system_opt['X'].tolist()
            Y0 = system_opt['Y'].tolist()
            N0 = system_opt['N'].tolist()

            # Initial conditions
            Jk = 0
            w=[]
            w0 = []
            lbw = []
            ubw = []
            g=[]
            lbg = []
            ubg = []

            # Constraints on input
            U = MX.sym("U",1)
            V = MX.sym("V",1)
            w += [U, V]
            lbw += [-inf]*2
            ubw += [inf]*2
            w0 += [0,util_bound[tk]]
            g   += [U**2 + V**2]
            lbg += [0]
            ubg += [util_bound[tk]**2]

            # Initial conditions
            Xk = MX.sym("Xk",nbuf)
            w += [Xk]
            lbw += X0
            ubw += X0
            w0 += X0
            Yk = MX.sym("Yk",nbuf)
            w += [Yk]
            lbw += Y0
            ubw += Y0
            w0 += Y0
            Nk = MX.sym("Nk",nbuf)
            w += [Nk]
            lbw += N0
            ubw += N0
            w0 += N0

            # Future values of x, y and n
            Xf = xs[tk:tk+horizon]
            Xf = Xf[::].tolist()
            Yf = ys[tk:tk+horizon]
            Yf = Yf[::].tolist()
            Nf = bs[tk:tk+horizon]
            Nf = Nf[::].tolist()

            # Apply the first input 
            Xk = F(Xk,Xf[0]+U)
            Yk = F(Yk,Yf[0]+V)
            Nk = F(Nk,Nf[0])
            Jt = J(x=Xk,y=Yk,nx=Nk)
            Jk += Jt['P']
            for k in range(horizon-1):
                # New NLP variable for the control
                Uk = MX.sym('U_' + str(k))
                Vk = MX.sym('V_' + str(k))
                w   += [Uk, Vk]
                lbw += [-inf]*2
                ubw += [inf]*2
                w0 += [0]*2

                g   += [Uk**2 + Vk**2]
                lbg += [0]
                ubg += [util_bound[tk+k+1]**2]

                Xk = F(Xk,Xf[k+1]+Uk)
                Yk = F(Yk,Yf[k+1]+Vk)
                Nk = F(Nk,Nf[k+1])
                Jt = J(x=Xk,y=Yk,nx=Nk)
                Jk += Jt['P']


            prob = {'f': -Jk, 'x': vertcat(*w), 'g': vertcat(*g)}
            solver = nlpsol('solver', 'ipopt', prob,{"ipopt": {"print_level": 5}});
            sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
            w_opt = sol['x'].full().flatten()
            J_opt = -sol['f']

            # Plot the solution
            u_opt = w_opt[0]
            v_opt = w_opt[1]
            U_mpc += [u_opt]
            V_mpc += [v_opt]
            x_mpc += [Xf[0] + u_opt]
            y_mpc += [Yf[0] + v_opt]
            system_opt = system_update(system_opt, Xf[0] + u_opt, Yf[0] + v_opt, 1)
            barxt, baryt, privt = privacy(system_opt)
            barx_mpc += [barxt] 
            bary_mpc += [baryt] 
            priv_mpc += [privt] 
            # Compute utility
            util_mpc += [utility(u_opt,v_opt)]
            time_mpc += [time.time()-time_0]
            #system_opt = system_update(system_opt, Xf[0] + u_opt, Yf[0] + v_opt, 1)

    clear_output(wait=False)

    data_mpc = pd.DataFrame()
    data_mpc['x'] = x_mpc
    data_mpc['y'] = y_mpc
    data_mpc['u_enu']= us[:len(x_mpc)]
    data_mpc['U'] = U_mpc
    data_mpc['V'] = V_mpc
    data_mpc['barx'] = barx_mpc
    data_mpc['bary'] = bary_mpc
    data_mpc['priv'] = priv_mpc
    data_mpc['util'] = util_mpc
    data_mpc['time'] = time_mpc

    return data_mpc#U_mpc, V_mpc, x_mpc, y_mpc, barx_mpc, bary_mpc, priv_mpc, util_mpc 

## Main function

In [6]:
def func_run(user_n,horizons,GeoI_filename,folder_sol):
    user= str(user_n) # Examples: abboip oilrag 51 90 14
    #Name your files
    #local_url="datasets/"+dataset+"-tree_93/"+user+".csv"
    sol_file='solutions/'+dataset+'_30s/'+folder_sol+'/sol_' +str(user_n)
    #sol_file='solutions/'+dataset+'_27072023/RF2/sol_' +str(user_n)
    file_GeoI='solutions/'+dataset+'_30s/'+GeoI_filename+'/sol_' +str(user_n)
    exists = os.path.isfile(file_GeoI+'/Data_GeoI.csv')
    if exists:
        #data = pd.read_csv(local_url,names=["latitude","longitude","timestamp"])
        print("uploaded from local file")
    else:
        return #if user doesn't exist return out of the function in order to get another user


    if not os.path.exists(sol_file):
        os.makedirs(sol_file)
        if not os.path.isfile('solutions/'+dataset+'_30s/'+folder_sol+'/info.txt'):
            text_file = open('solutions/'+dataset+'_30s/'+folder_sol+'/info.txt', "w")
            ox = text_file.write('GeoI folder used : '+GeoI_filename)
            text_file.close()
        
        
    Data_GeoI=pd.read_csv(file_GeoI+'/Data_GeoI.csv')    
    Data_real=pd.read_csv(file_GeoI+'/Data_real.csv')
    xs=Data_real['x']
    ys=Data_real['y'] 
    bs=Data_real['b']
    us=Data_real['u_enu']
    util=Data_GeoI['util'] 
    util2=Data_GeoI.loc[Data_GeoI['util']>0].mean()
    
    start_time = time.monotonic()
    Data_MPC = {} # Empty dictionary
    for h in horizons:
        data_mpc = solve_mpc(xs,ys,us,bs,h,nbuf,util)
        Data_MPC[h] = data_mpc
        end_time = time.monotonic()
        print('user',user,' horizon ',h,' runtime ',timedelta(seconds=end_time - start_time), ' total time ',timedelta(seconds=end_time - time_0))
        Data_MPC[h].to_csv(sol_file+'\\save_data_mpc_horizon_' + str(h) + '.csv')


    end_time = time.monotonic()
    print(timedelta(seconds=end_time - start_time))



    # Save data MPC
    pickle.dump( horizons, open( sol_file+"/Horizons_MPC.pkl", "wb" ) )
    pd.DataFrame(horizons).to_csv(sol_file+"/horizons.csv") 
    pickle.dump( Data_MPC, open( sol_file+"/Data_MPC.pkl", "wb" ) )
    
    

## Run MPC

In [ ]:
dataset="privamov" # OPTIONS ARE: cabspotting privamov
#horizon 7->3min
hor=[1,2,3,4,5,6,7]#,8,9,10]
#hor=[16]#,2,3,5,8,13,16]#[1,2,3,5,8]#[13,16]#[1,2,3,5,8,13,16]
time_0 = time.monotonic()
for user in range(1,117):
    func_run(user,hor,'GeoI_10000','RF_10000')

user 4  horizon  5  runtime  0:19:31.282000  total time  1:14:43.313000
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       78

Total number of variables............................:       12
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        6
        inequality constraints with only upper bounds:        0

iter    objecti

  18 -4.0672392e+002 0.00e+000 2.04e-002  -1.7 3.70e+002  -4.5 1.00e+000 1.00e+000h  1
  19 -4.2737863e+002 6.69e-001 3.70e-002  -1.7 8.84e+002  -5.0 1.00e+000 1.00e+000f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20 -4.4016390e+002 9.56e+001 3.10e-002  -1.7 2.01e+003  -5.5 1.00e+000 3.31e-001f  1
  21 -4.4087421e+002 1.00e+002 3.05e-002  -1.7 2.96e+003  -5.9 1.00e+000 1.76e-002h  1
  22 -4.8692037e+002 1.22e+003 1.73e-001  -1.7 4.64e+003  -6.4 1.00e+000 7.10e-001f  1
  23 -4.7561767e+002 5.58e+001 2.07e-002  -1.7 8.67e+000  -2.4 1.00e+000 1.00e+000h  1
  24 -4.7509461e+002 0.00e+000 4.49e-003  -1.7 3.15e+000  -2.8 1.00e+000 1.00e+000h  1
  25 -4.7516364e+002 0.00e+000 4.49e-003  -1.7 9.44e+000  -3.3 1.00e+000 1.00e+000h  1
  26 -4.7538142e+002 0.00e+000 4.48e-003  -1.7 2.83e+001  -3.8 1.00e+000 1.00e+000h  1
  27 -4.7600761e+002 0.00e+000 4.45e-003  -1.7 8.43e+001  -4.3 1.00e+000 1.00e+000h  1
  28 -4.7782502e+002 0.00e+000 4.37e-003  -1.7 2.

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       78

Total number of variables............................:       12
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        3
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        3
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

   8 -4.8145548e+002 0.00e+000 2.13e-001  -1.7 8.71e-001  -0.6 7.63e-001 1.00e+000f  1
   9 -4.8192773e+002 0.00e+000 1.51e-001  -1.7 1.94e+000  -1.1 1.00e+000 1.00e+000f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10 -4.8258501e+002 0.00e+000 1.10e-001  -1.7 3.92e+000  -1.6 6.08e-001 1.00e+000h  1
  11 -4.8365953e+002 0.00e+000 1.10e-001  -1.7 8.38e+000  -2.1 1.00e+000 1.00e+000h  1
  12 -4.8546170e+002 0.00e+000 7.14e-002  -1.7 1.75e+001  -2.5 6.17e-001 1.00e+000h  1
  13 -4.8828624e+002 9.43e+001 9.51e-002  -1.7 3.66e+001  -3.0 1.00e+000 1.00e+000h  1
  14 -4.9070056e+002 1.03e+002 6.50e-002  -1.7 7.66e+001  -3.5 6.21e-001 8.07e-001h  1
  15 -4.9482277e+002 6.55e+001 1.15e-001  -1.7 1.57e+002  -4.0 1.00e+000 1.00e+000h  1
  16 -7.2780949e+002 5.24e+004 1.85e+000  -1.7 3.73e+002  -4.4 5.45e-001 1.00e+000h  1
  17 -5.8154786e+002 1.18e+004 2.97e-001  -1.7 8.09e+002  -4.9 1.00e+000 1.00e+000h  1
  18 -5.3853050e+002 2.13e+003 3.98e-002  -1.7 1.

  19 -1.0136437e-006 1.45e+001 1.60e-009  -8.6 3.88e+000  -8.8 1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20 -9.5748267e-007 3.04e-001 1.88e-010  -8.6 9.37e-001  -9.3 1.00e+000 1.00e+000h  1
  21 -9.5374896e-007 0.00e+000 5.12e-012  -8.6 6.22e-002  -9.7 1.00e+000 1.00e+000h  1

Number of Iterations....: 21

                                   (scaled)                 (unscaled)
Objective...............: -9.5374896285960484e-007  -9.5374896285960484e-007
Dual infeasibility......:  5.1248352277089772e-012   5.1248352277089772e-012
Constraint violation....:  0.0000000000000000e+000   0.0000000000000000e+000
Complementarity.........:  2.5068482878740514e-009   2.5068482878740514e-009
Overall NLP error.......:  2.5068482878740514e-009   2.5068482878740514e-009


Number of objective function evaluations             = 22
Number of objective gradient evaluations             = 22
Number of equality constraint evaluations            

...

  34 -4.2896611e+002 2.91e+003 2.15e-002  -2.5 2.73e+003    -  1.00e+000 9.02e-001h  1
  35 -4.2580972e+002 3.30e+002 1.42e-002  -2.5 1.34e+001  -2.7 1.00e+000 1.00e+000h  1
  36 -4.2585443e+002 1.06e+002 1.81e-003  -2.5 1.03e+001    -  1.00e+000 1.00e+000h  1
  37 -4.2637294e+002 1.07e+003 4.19e-003  -2.5 2.82e+001    -  1.00e+000 1.00e+000h  1
  38 -4.2571062e+002 6.84e+001 1.44e-004  -2.5 7.88e+000    -  1.00e+000 1.00e+000h  1
  39 -4.2567444e+002 0.00e+000 1.47e-005  -2.5 2.15e+000    -  1.00e+000 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  40 -4.2567045e+002 0.00e+000 5.75e-008  -2.5 1.07e-001    -  1.00e+000 1.00e+000h  1
  41 -4.2568647e+002 0.00e+000 8.89e-008  -3.8 8.79e+000    -  1.00e+000 1.00e+000h  1
  42 -4.2568648e+002 0.00e+000 3.43e-013  -3.8 3.23e-002    -  1.00e+000 1.00e+000h  1
  43 -4.2568738e+002 0.00e+000 2.72e-010  -5.7 4.91e-001    -  1.00e+000 1.00e+000f  1
  44 -4.2568738e+002 0.00e+000 5.55e-017  -5.7 1.

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       78

Total number of variables............................:       12
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        6
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

  17 -3.9427327e+002 0.00e+000 1.37e-002  -2.5 1.38e+001  -3.1 6.84e-001 1.00e+000h  1
  18 -3.9570748e+002 2.28e+002 1.64e-002  -2.5 2.89e+001  -3.6 7.00e-001 1.00e+000h  1
  19 -3.9597066e+002 2.21e+002 1.59e-002  -2.5 1.48e+002    -  3.31e-001 8.75e-002h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20 -3.9995402e+002 3.99e+002 1.32e-001  -2.5 1.82e+002    -  8.67e-001 1.00e+000h  1
  21 -3.9815266e+002 1.06e+001 1.11e-001  -2.5 2.79e+000  -1.4 1.00e+000 1.00e+000h  1
  22 -4.0031580e+002 4.64e+000 3.68e-002  -2.5 3.29e+002    -  7.58e-001 6.73e-001h  1
  23 -3.9998346e+002 0.00e+000 2.02e-002  -2.5 1.40e+000  -1.8 1.00e+000 1.00e+000h  1
  24 -4.0795653e+002 2.18e+002 1.29e-001  -2.5 8.92e+002    -  5.23e-001 1.00e+000f  1
  25 -4.0567024e+002 3.02e+000 8.69e-003  -2.5 5.69e+000  -2.3 1.00e+000 1.00e+000h  1
  26 -4.1387497e+002 5.86e+002 2.92e-002  -2.5 2.96e+003    -  4.55e-001 5.37e-001f  1
  27 -4.1218061e+002 4.22e+001 5.97e-003  -2.5 7.

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       78

Total number of variables............................:       12
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        6
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

   4 -3.8686850e+002 0.00e+000 1.90e+000  -1.0 1.56e-003   3.1 9.91e-001 1.00e+000h  1
   5 -3.8687029e+002 0.00e+000 1.78e+000  -1.0 4.42e-003   2.6 1.00e+000 1.00e+000f  1
   6 -3.8687506e+002 0.00e+000 1.52e+000  -1.0 1.17e-002   2.1 1.00e+000 1.00e+000f  1
   7 -3.8688611e+002 0.00e+000 1.06e+000  -1.0 2.70e-002   1.7 1.00e+000 1.00e+000f  1
   8 -3.8690863e+002 0.00e+000 5.88e-001  -1.0 5.52e-002   1.2 1.00e+000 1.00e+000f  1
   9 -3.8691824e+002 0.00e+000 1.54e-001  -1.7 2.62e-002   0.7 9.49e-001 1.00e+000h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10 -3.8693762e+002 0.00e+000 1.17e-001  -2.5 6.07e-002   0.2 1.00e+000 1.00e+000h  1
  11 -3.8698827e+002 0.00e+000 7.01e-002  -2.5 1.44e-001  -0.3 7.30e-001 1.00e+000h  1
  12 -3.8708289e+002 0.00e+000 6.75e-002  -2.5 2.94e-001  -0.7 7.76e-001 1.00e+000h  1
  13 -3.8722990e+002 3.08e+000 6.53e-002  -2.5 1.39e+003    -  2.61e-001 3.21e-002f  1
  14 -3.9192214e+002 3.16e+003 1.51e-001  -2.5 5.